### *****Make sure to change the .csv file name in two places before running.*****
### Look for: # <-------------Change to: df_<Your File #>

In [ ]:
#Import packages
import pandas as pd
import requests
import json
import numpy as np
import time
import gmaps

from config import (mqkey, gkey) 

In [ ]:
#load crash data
#file_path = 'Resources/CRASH DATA SET'

#crash_csv = pd.read_csv(file_path)

#place_holder_df = pd.DataFrame()

In [ ]:
#set list of lat lng to feed into google

#lat = df['Start_lat']
#lng = df['Start_lng']

#for row in crash_csv:
    #lat = df['Start_lat']
    #lng = df['Start_lng']
    #google_path = f'{lat},{lng}'
    #params = {'Key': gkey,
         #'Path' : google_path}
    #base_url = 'https://roads.googleapis.com/v1/speedLimits?'

    #response = requests.get(base_url, params=params)
    
    #roads_data = response.json()
    
    
    

In [ ]:
# convert to json
#roads_data = response.json()



In [ ]:
# Jon's code starts here.

### Make sure to change the .csv file name in two places before running.
### Look for: # <-------------Change to: df_<Your File #>

In [ ]:
# File to Load
file_to_load = "Resources/df_2.csv" # <-------------Change to: df_<Your File #>

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file_to_load)

#print(df)

# Add "Speed Limit" and "Nearest Intersection" columns to the DataFrame.
df["Speed Limit"] = ""
df["Nearest Intersection"] = ""
#df

In [ ]:
# Set parameters for speed limit search.
params = {
    "key": mqkey,
    "includeRoadMetadata": "true",
    "includeNearestIntersection": "true"
}

# Set a variable to track each iteration.
Request_number = 0

# Use for loop to locate the speed limits.
for index, row in df.iterrows():
    
    # Get Lat & Lng from DataFrame.
    lat = row["Start_Lat"]
    lng = row["Start_Lng"]
    
    # Change location each iteration while leaving original params in place.
    #params["location"] = f'{lat},{lng}'
    location = f"{lat},{lng}"
    
    # Set base URL.
    base_url = "http://www.mapquestapi.com/geocoding/v1/reverse?includeRoadMetadata=true&includeNearestIntersection=true"
    query_url = f"{base_url}&key={mqkey}&location={location}"
    
    
    # Make requests and print URL.
    speed = requests.get(query_url)
    #print(speed.url)
    
    # Convert to json
    speed = speed.json()
    
    # In case there may be missing speed limits, use try-except to skip any missing data.
    try:
        df.loc[index, "Speed Limit"] = speed["results"][0]["locations"][0]["roadMetadata"]["speedLimit"]
        df.loc[index, "Nearest Intersection"] = speed["results"][0]["locations"][0]["nearestIntersection"]["streetDisplayName"]
        Request_number = Request_number + 1
        #time.sleep(.1)
        print(f"Processed request number {Request_number}")
        #if Request_number == 10:
            #break

    except Exception as e:
        Request_number = Request_number + 1
        print("Speed limit or Nearest Intersection not found... Next")
        print(e)
        #if Request_number == 10:
            #break
        
        pass  

In [ ]:
df

In [ ]:
# Count how many rows were blank in 'Speed Limit' column.
len(df[df['Speed Limit'] == ''])

In [ ]:
# Count how many rows were blank in 'Nearest Intersection' column.
len(df[df['Nearest Intersection'] == ''])

### Make sure to change the .csv file name in two places before running.
### Look for: # <-------------Change to: df_<Your File #>

In [ ]:
# Save to .csv
df.to_csv(r'Resources\df_2WithSpeed.csv', index=False) # <-------------Change to: df_<Your File #>

In [ ]:
#Jon's code ends here.

In [ ]:
#Calvin's Heatmap

In [ ]:
# File to Load
file_to_load = "Resources/AccidentDataWithSpeed.csv" # <-------------Change to: df_<Your File #>

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file_to_load)
df.head()

In [ ]:
 # Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [ ]:
 # Store 'Lat' and 'Lng' into  locations 
locations = df[["Start_Lat", "Start_Lng"]].astype(float)

In [ ]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=df["Severity"], 
                                 dissipating=False, max_intensity=4
                                 
                                 
                                 ,
                                 point_radius = .02)
fig.add_layer(heat_layer)

fig